In [ ]:
import os
import ase.io
from typing import Optional, List

import numpy as np
from matplotlib import pyplot as plt
import MDAnalysis as mda

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [ ]:
root = "results/md17/mace-salicylic-50k-exp1"

In [ ]:
### Load all test results from folder ###
#########################################

test_systems_data = []
for filename in os.listdir(root):
    if not filename.startswith("ds_"):
        continue
    if not filename.endswith("_target.xyz"):
        continue
    pred_frames = ase.io.read(os.path.join(root, filename[:-11] + ".xyz"), index=":", format="extxyz")
    target_frames = ase.io.read(os.path.join(root, filename), index=":", format="extxyz")
    name = filename.split('__')[1]
    system_data = []
    for pred, target in zip(pred_frames, target_frames):
        frame_data = {
        'name': name,
        'coords': pred.arrays['positions'],
        'atom_types': pred.arrays['numbers'],
        'forces_pred': pred.arrays['forces'],
        'forces_target': target.arrays['forces'],
        'energies_pred': pred.info["energy"],
        'energies_target': target.info["energy"],
        }
        system_data.append(frame_data)
    if len(system_data) == 0:
        frame_data = {
            'name': name,
        }
        system_data.append(frame_data)
    test_systems_data.append(system_data)

In [ ]:
forces_means = []
energies_means = []
for data in test_systems_data[0]:
    forces_means.append(np.abs(data['forces_pred'] - data['forces_target']).mean())
    energies_means.append(np.abs(data['energies_pred'] - data['energies_target']).mean())
forces_means = np.array(forces_means)
energies_means = np.array(energies_means)

In [ ]:
fm = forces_means.mean()
em = energies_means.mean()

fm, em, fm * 43, em * 43 # kcal/mol | meV